# Clean Stripe and identify plan duration using CM data

###Specifications

####  Stripe:

o Stripe- Add a new column to Stripe data and then perform the check with CM Stripe data to identify each item by Mon or Annual. There are few non-revenue Stripe data like fees /disputes. Move these to the bottom before doing this.


### Script
---

#### Imports,prepartions and functions

In [1]:
import pandas as pd
import os
!pip install XlsxWriter
import xlsxwriter


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 3.6 MB/s eta 0:00:00


###### Preparations
---


In [2]:
#Mount GDrive
from google.colab import drive
drive.mount('/content/gdrive')

#Set file paths
path = "/content/gdrive/My Drive/Data Science 2022/CM Audit/"

filestripe = path + "Stripe Data/2023/Stripe Raw Data - 2023.02.xlsx"
filecm  =path + "Monthly Output/2023.02/CM_2023.02.xlsx"

outfile = path + "Monthly Output/2023.02/Stripe_2023.02.xlsx"


Mounted at /content/gdrive


#### Cleanup Stripe and  Compare with CM data to identify if the charges are annual or monthly

In [3]:
lstType = ['dispute','dispute_reversal','fee']

#Load file
if filestripe[-3:] == "csv": 
  month = filestripe[-9:-4]
  df = pd.read_csv(filestripe , na_filter=False, index_col=False)
else:
  month = filestripe[-10:-5]
  df = pd.read_excel(filestripe , na_filter=False, index_col=False)

############ Segregate Non-Revenue transactions based on 'Type'
#Move the non revenue 'reporting_category' to seperate sheet - Non revenue

dfmove = df[df['reporting_category'].isin(lstType)] #dataset to be moved to 'others' sheet
dfmove.insert(dfmove.shape[1],"Plan Type","")
dfclean = df[~df['reporting_category'].isin(lstType)] #clean dataset

dfs = dfclean.copy()

dfcm = pd.read_excel(filecm,sheet_name="Stripe" , na_filter=False, index_col=False)

dfmon = pd.merge(dfs, dfcm, right_on="Customer External ID", left_on="customer_id", how='left')

#Keep only stripe data along with plan
cmcols = dfcm.columns.tolist()
cmcols.remove('Plan Type')

mcols = dfmon.columns.tolist()
newcols = [x for x  in mcols if x not in cmcols  ]
dfnews = dfmon[newcols]

#Delete duplicate rows
dfnews.drop_duplicates(inplace=True)
print(dfnews.shape)
#Look for duplicate ChargeIDs. 
lstduplicate = dfnews[dfnews.duplicated('source_id')].source_id.to_list()
#For those chargeIDs find corresponding CM record with "Line Item Type" = 'subscription'. Get 'plan type' from that transaction
#Get all records from CM that has chargeid = dfduplicate['source_id']
dfcm_match = dfcm[dfcm['Charge ID'].isin(lstduplicate)]
dfcm_subs = dfcm_match[dfcm_match["Line Item Type"] == 'subscription']
#Iterate cm data to find plan type
lsthit=[]
for i,row in dfcm_subs.iterrows():
  for item in lstduplicate:
    if row['Charge ID'] == item:
      dfnews.loc[dfnews['source_id'] == item, ['Plan Type']] = row['Plan Type'] 
      lsthit.append(item)
#There are records with same customer id repeated twice. Sort out those
lstcus_duplicates =[ x for x in lstduplicate  if x not in lsthit  ]

#Remove those records from dfnews where chargeID is not in lsthit
for item in lstcus_duplicates:
  dfnews.drop(dfnews.index[(dfnews['source_id'] == item) & (dfnews['Plan Type'] == 'Yearly')],inplace=True)

dfnews.drop_duplicates(inplace=True)

#Sort by plan type
dfnews = dfnews.sort_values('Plan Type',ascending=False)

writer = pd.ExcelWriter(outfile, engine='xlsxwriter')
dfnews.to_excel(writer,sheet_name="Cleaned Data", index=False)
dfmove.to_excel(writer,sheet_name="Non Revenue records", index=False)

writer.save()
writer.close()

<ipython-input-3-197bc95b09dd>:9: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  df = pd.read_excel(filestripe , na_filter=False, index_col=False)
/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.

(40917, 66)


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/usr/local/lib/python3.8/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
